# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

**Открываем и изучаем первые строки**

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


**Выводим инфо**

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


**Проверяем на дубликаты**

In [4]:
df.duplicated().sum()

0

**Меняем типы данных и проверяем**

In [5]:
df['calls']=df['calls'].astype("int")
df['messages']=df['messages'].astype("int")

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int64  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 125.7 KB


**Описание данных**
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

**Вывод:**
- Пропусков нет
- Дубликанов нет
- Типы данных корректны

## Разбейте данные на выборки

**Исходные данные разбивают в соотношении 3:1:1**
- Обучающая выборка (features_train, target_train) 60%
- Валидационая выборка (features_valid, target_valid) 20%
- Тестовая выборка (features_test, target_test) 20%

In [7]:
df_train, df_test = train_test_split(df, test_size=0.4, random_state=12345)
features = df_test.drop('is_ultra', axis=1)
target = df_test['is_ultra']
features_valid, features_test, target_valid, target_test = train_test_split(features, target, test_size=0.5, random_state=12345)
features_train = df_train.drop('is_ultra', axis=1)
target_train = df_train['is_ultra']

## Исследуйте модели

**Смена гиперпараметров**

Самый важный гиперпараметр решающего дерева — max_depth.
Настроим его для нашего дерева решений.

In [8]:
for depth in range(1, 10):
    model = DecisionTreeClassifier(random_state=12345, max_depth = depth) 

    model.fit(features_train,target_train) 

    predictions_valid = model.predict(features_valid) 

    result = accuracy_score(target_valid, predictions_valid) 

    print('max_depth =', depth, ':', result)

max_depth = 1 : 0.7542768273716952
max_depth = 2 : 0.7822706065318819
max_depth = 3 : 0.7853810264385692
max_depth = 4 : 0.7791601866251944
max_depth = 5 : 0.7791601866251944
max_depth = 6 : 0.7838258164852255
max_depth = 7 : 0.7822706065318819
max_depth = 8 : 0.7791601866251944
max_depth = 9 : 0.7822706065318819


**Вывод: лучший параметр max_depth = 3 : 0.7853810264385692**

**Дерево решений**

In [9]:
best_model_dt = None
best_result = 0
best_depth = 0

for depth in range(1, 4):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train,target_train)
    predictions_valid = model.predict(features_valid) 
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model_dt = model
        best_result = result
        best_depth = depth

print("Лучшая модель:", best_result, "Глубина дерева:", best_depth)

Лучшая модель: 0.7853810264385692 Глубина дерева: 3


**Случайный лес**

In [10]:
best_model_rf = None
best_result = 0
for est in range(1, 10):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(features_train,target_train) 
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model_rf = model
        best_result = result
        best_depth = depth

print("Лучшая модель:", best_result, "Глубина дерева:", best_depth)

Лучшая модель: 0.7822706065318819 Глубина дерева: 3


**Логистическая регрессия**

In [11]:
model = LogisticRegression(random_state=12345, solver='lbfgs')
model.fit(features_train, target_train)
result = model.score(features_valid, target_valid)
print("Качество модели:", result)

Качество модели: 0.7107309486780715


## Проверьте модель на тестовой выборке

**Наиболее качественные результаты, после проверки точности на валидной выборке, прогнозирует модель "Дерево решений", проверим ее на тестовой выборке, указав лучший гиперпараметр max_depth=3**

In [17]:
model = DecisionTreeClassifier(max_depth=3, random_state=12345)
model.fit(features_train, target_train)
result = model.score(features_test, target_test)
print("Качество модели:", result)

Качество модели: 0.7791601866251944


In [18]:
best_model_rf

RandomForestClassifier(n_estimators=8, random_state=12345)

### Вывод

**Были проверены три модели: дерево решений, случайный лес и логистическая регрессия. После проверки точности этих моделей на валидной выборке, можно прийти к выводу, что в данном случае наиболее качественные результаты прогнозирует модель "Дерево решений", этот результат был подтвержден после на тестовой выборке**

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
